In [ ]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt

### 1. 데이터 준비

In [ ]:
# === 1. 데이터 불러오기 ===
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/[2025-1학기] 데이터사이언스/Data1_Train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/[2025-1학기] 데이터사이언스/Data1_Test.csv')  # test set은 2020~2025년 데이터라고 가정

In [ ]:
# === 2. 특성과 타겟 분리 ===
# One-Hot Encoding된 MONTH 열들 찾기
month_cols = [col for col in train_df.columns if col.startswith('MONTH_')]
# 입력 변수
X_all = train_df[['YEAR']+ month_cols]
# 출력 변수
y_all = train_df[['MAX_TEMP', 'MIN_TEMP', 'temp_range', 'AVG_WIND_SPEED', 'daily_precip']]

# === 3. 학습/검증 데이터 분할 ===
X_train = X_all[train_df['YEAR'] <= 2014]
y_train = y_all[train_df['YEAR'] <= 2014]

X_val = X_all[(train_df['YEAR'] >= 2015)]
y_val = y_all[(train_df['YEAR'] >= 2015)]

# # === 4. 테스트 데이터 준비 ===
# X_test = test_df[['YEAR']+ month_cols]
# y_test = test_df[['MAX_TEMP', 'MIN_TEMP', 'temp_range', 'AVG_WIND_SPEED', 'daily_precip']]

### 2. 타깃 변수 설정

In [ ]:
# 타깃 변수들의 분산과 표준편차 계산
target_vars = ['MAX_TEMP', 'MIN_TEMP', 'temp_range', 'AVG_WIND_SPEED', 'daily_precip']

for col in target_vars:
    var = train_df[col].var()
    std = train_df[col].std()
    print(f"{col} - 분산: {var:.4f}, 표준편차: {std:.4f}")


MAX_TEMP - 분산: 77.5180, 표준편차: 8.8044
MIN_TEMP - 분산: 88.4402, 표준편차: 9.4043
temp_range - 분산: 3.4640, 표준편차: 1.8612
AVG_WIND_SPEED - 분산: 0.1996, 표준편차: 0.4468
daily_precip - 분산: 8.8329, 표준편차: 2.9720


In [ ]:
# 이상치 수 확인 (IQR 방법)
def count_outliers(series):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = series[(series < lower_bound) | (series > upper_bound)]
    return len(outliers)

for col in target_vars:
    num_outliers = count_outliers(train_df[col])
    print(f"{col} - 이상치 개수: {num_outliers}")


MAX_TEMP - 이상치 개수: 0
MIN_TEMP - 이상치 개수: 0
temp_range - 이상치 개수: 0
AVG_WIND_SPEED - 이상치 개수: 1
daily_precip - 이상치 개수: 27


### XGBRegressor

In [ ]:
pip install scikit-optimize

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 2.7 MB/s eta 0:00:00


In [ ]:
from xgboost import XGBRegressor
from skopt import BayesSearchCV
from skopt.space import Real, Integer
from sklearn.model_selection import train_test_split

# 타겟 중 하나만 분리
y_train_single = y_train['MAX_TEMP']

# 탐색할 파라미터 범위 정의
param_space = {
    'n_estimators': Integer(100, 1000),
    'max_depth': Integer(3, 10),
    'learning_rate': Real(0.01, 0.3, prior='log-uniform'),
    'subsample': Real(0.6, 1.0),
    'colsample_bytree': Real(0.6, 1.0),
    'reg_alpha': Real(0, 1),
    'reg_lambda': Real(0.5, 3),
}

# 기본 모델
xgb = XGBRegressor(random_state=42)

# BayesSearchCV 설정
opt = BayesSearchCV(
    estimator=xgb,
    search_spaces=param_space,
    n_iter=30,  # 30번 탐색, 필요하면 늘리기
    cv=3,
    scoring='neg_mean_absolute_error',  # MAE 기준으로 최적화
    random_state=42,
    n_jobs=-1,
    verbose=0
)

# 튜닝 실행
opt.fit(X_train, y_train_single)

# 최적 파라미터 출력
print("Best params:", opt.best_params_)

# 최적 파라미터로 MultiOutputRegressor 생성
from sklearn.multioutput import MultiOutputRegressor

best_params = opt.best_params_
best_xgb = XGBRegressor(**best_params, random_state=42)
model = MultiOutputRegressor(best_xgb)
model.fit(X_train, y_train)

Best params: OrderedDict([('colsample_bytree', 0.6), ('learning_rate', 0.01), ('max_depth', 3), ('n_estimators', 1000), ('reg_alpha', 0.0), ('reg_lambda', 0.863264718233512), ('subsample', 0.6)])


MultiOutputRegressor(estimator=XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=0.6, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=None,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=0.01, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=3,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=1000, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=42, ...))

In [ ]:
# === 6. 예측 ===
val_pred = model.predict(X_val)
# test_pred = model.predict(X_test)

# === 7. 평가 함수 ===
def evaluate(true, pred, dataset_name):
    mae = mean_absolute_error(true, pred)
    mse = mean_squared_error(true, pred)
    rmse = mse ** 0.5
    r2 = r2_score(true, pred)

    # 역방향 지표로 정렬을 통일 (작을수록 좋음)
    adjusted_r2 = 1 - r2

    print(f"📊 [{dataset_name} 성능 평가]")
    print(f"MAE : {mae:.4f}")
    print(f"MSE : {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"R²  : {r2:.4f} (그래프에선 1 - R² = {adjusted_r2:.4f})\n")

    return {
        'MAE': mae,
        'MSE': mse,
        'RMSE': rmse,
        '1 - R²': adjusted_r2
    }

# === 8. 성능 평가 ===
evaluate(y_val, val_pred, "Validation")
# evaluate(y_test, test_pred, "Test")

📊 [Validation 성능 평가]
MAE : 0.9272
MSE : 1.7990
RMSE: 1.3413
R²  : 0.7199 (그래프에선 1 - R² = 0.2801)



{'MAE': 0.9272078275680542,
 'MSE': 1.7990305423736572,
 'RMSE': 1.3412794423138146,
 '1 - R²': 0.2800617814064026}

### CatBoostRegressor

In [ ]:
pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 6.0 MB/s eta 0:00:00


In [ ]:
pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 10.5 MB/s eta 0:00:00


In [ ]:
import optuna
import numpy as np
import pandas as pd
from catboost import CatBoostRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split


target_cols = ['MAX_TEMP', 'MIN_TEMP', 'temp_range', 'AVG_WIND_SPEED', 'daily_precip']

# Optuna 목적 함수 정의 (전체 타깃 MAE 평균 최소화)
def objective(trial):
    params = {
        'iterations': trial.suggest_int('iterations', 500, 2000),
        'depth': trial.suggest_int('depth', 4, 10),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10),
        'random_seed': 42,
        'verbose': 0,
    }

    # CatBoostRegressor 기본 생성기
    cb = CatBoostRegressor(**params)
    model = MultiOutputRegressor(cb)

    model.fit(X_train, y_train)

    val_pred = model.predict(X_val)

    # 전체 타깃 MAE 평균 계산
    mae_list = []
    for i in range(len(target_cols)):
        mae = mean_absolute_error(y_val.iloc[:, i], val_pred[:, i])
        mae_list.append(mae)
    mean_mae = np.mean(mae_list)

    return mean_mae

# Optuna 스터디 생성 및 최적화
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=30)  # trial 수 조절 가능

print("Best params:", study.best_params)
print("Best MAE:", study.best_value)

# 최적 파라미터로 모델 재학습
best_params = study.best_params
best_params['random_seed'] = 42
best_params['verbose'] = 0
final_cb = CatBoostRegressor(**best_params)
final_model = MultiOutputRegressor(final_cb)
final_model.fit(X_train, y_train)

# 평가
val_pred = final_model.predict(X_val)
# test_pred = final_model.predict(X_test)

def evaluate(true, pred, dataset_name):
    mae = mean_absolute_error(true, pred)
    mse = mean_squared_error(true, pred)
    rmse = mse ** 0.5
    r2 = r2_score(true, pred)

    # 역방향 지표로 정렬을 통일 (작을수록 좋음)
    adjusted_r2 = 1 - r2

    print(f"📊 [{dataset_name} 성능 평가]")
    print(f"MAE : {mae:.4f}")
    print(f"MSE : {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"R²  : {r2:.4f} (그래프에선 1 - R² = {adjusted_r2:.4f})\n")

    return {
        'MAE': mae,
        'MSE': mse,
        'RMSE': rmse,
        '1 - R²': adjusted_r2
    }


evaluate(y_val.values, val_pred, "Validation")
# evaluate(y_test.values, test_pred, "Test")

[I 2025-05-28 03:41:37,098] A new study created in memory with name: no-name-d3d33f4b-eb5a-4bdf-a35f-3c683a59bf6e
<ipython-input-20-cf3633fcb656>:17: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
[I 2025-05-28 03:41:40,294] Trial 0 finished with value: 1.1355793592696484 and parameters: {'iterations': 1894, 'depth': 6, 'learning_rate': 0.2523039925322094, 'l2_leaf_reg': 9.153954932417708}. Best is trial 0 with value: 1.1355793592696484.
<ipython-input-20-cf3633fcb656>:17: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
[I 2025-05-28

Best params: {'iterations': 1044, 'depth': 5, 'learning_rate': 0.017562091311482585, 'l2_leaf_reg': 7.535013979351431}
Best MAE: 0.9018082318717575
📊 [Validation 성능 평가]
MAE : 0.9018
MSE : 1.6808
RMSE: 1.2965
R²  : 0.7155 (그래프에선 1 - R² = 0.2845)



{'MAE': 0.9018082318717575,
 'MSE': 1.6808370858442785,
 'RMSE': 1.2964710123424583,
 '1 - R²': 0.2844656647923538}